In [ ]:
!gdown --id 1jcd9ocuh50lnwrDSiY10FbYu5X6mDpCJ

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1jcd9ocuh50lnwrDSiY10FbYu5X6mDpCJ
To: /content/extracted_triples_2.json
100% 97.2k/97.2k [00:00<00:00, 64.6MB/s]


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!pip install pandas

In [ ]:
import json
import pandas as pd
import spacy

In [ ]:
nlp =  spacy.load('en_core_web_md')

#Правила

In [ ]:
correct_types = {
    'LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'],'correct':{'subj': ['ORG','MISC','LOC'],'obj':'LOC'}},
    'INCEPTION': {'subj': ['ORG','MISC','TIME','LOC'],'obj':['ORG','MISC','TIME','LOC'], 'correct':{'subj': ['ORG','MISC','LOC'],'obj':'TIME'}},
    'LANGUAGES_SPOKEN_WRITTEN_OR_SIGNED': {"subj": ['PER','MISC','LOC'], 'obj': ['PER','MISC', 'LOC'],'correct':{'subj': 'PER','obj':['MISC','LOC']}},
    'COUNTRY': {"subj": ['LOC','MISC','ORG','PER'],'obj':['LOC','MISC','ORG','PER'], 'correct':{'subj': ['LOC','MISC','ORG','PER'],'obj':'LOC'}},
    'CHAIRPERSON': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'ORG','obj':'PER'}},
    'MEMBER_OF_POLITICAL_PARTY': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':'ORG'}},
    'HEAD_OF_GOVERNMENT': {"subj": ['PER','LOC','ORG'], 'obj': ['PER','LOC','ORG'], 'correct':{'subj':['LOC','ORG'] ,'obj':'PER'}},
    'DATE_OF_BIRTH': {"subj": ['PER','TIME'], 'obj': ['PER','TIME'],'correct':{'subj':'PER' ,'obj':'TIME'}},
    'DATE_OF_DEATH': {"subj": ['PER','TIME'], 'obj': ['PER','TIME'],'correct':{'subj':'PER' ,'obj':'TIME'}},
    'SIBLING': {"subj": ['PER'], 'obj': ['PER'],'correct':{'subj':'PER' ,'obj':'PER'}},
    'PLACE_OF_BIRTH': {'subj': ['LOC','PER''ORG'],'obj':['LOC','PER','ORG'],'correct':{'subj': 'PER','obj':['LOC','ORG']}},
    'PLACE_OF_DEATH': {'subj': ['LOC','PER''ORG'],'obj':['LOC','PER''ORG'],'correct':{'subj': 'PER','obj':['LOC','ORG']}},
    'COUNTRY_OF_CITIZENSHIP': {"subj": ["PER", "LOC", "ORG", "MISC"], 'obj': ["PER", "LOC", "ORG", "MISC"], 'correct':{'subj': ["PER","ORG", "MISC"],'obj':'LOC'}},
    'POSITION_HELD': {"subj": ['PER','ORG','MISC','ORG'], 'obj': ['PER','ORG','MISC','ORG'],'correct':{'subj': 'PER','obj': ['ORG','MISC','ORG']}},
    'PARTICIPANT': {'subj': ['PER','ORG','LOC','MISC'],'obj': ['PER','ORG','LOC','MISC'], 'correct':{'subj': ['PER','ORG','LOC'],'obj':'MISC'}},
    'AUTHOR': {"subj": ['MISC','PER','ORG'], 'obj': ['MISC','PER','ORG'], 'correct':{'subj': 'MISC','obj':['PER','ORG']}},
    'DISSOLVED_ABOLISHED_OR_DEMOLISHED': {"subj": ['TIME','ORG','LOC','MISC'], 'obj': ['TIME','ORG','LOC','MISC'],'correct':{'subj': ['ORG','LOC','MISC'],'obj': 'TIME'}},
    'CONTAINS_ADMINISTRATIVE_TERRITORIAL_ENTITY' : {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': 'LOC','obj':['LOC','ORG']}},
    'HEADQUARTERS_LOCATION': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': 'ORG','obj':'LOC'}},
    'PUBLICATION_DATE': {"subj": ['TIME','MISC'], 'obj': ['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'NOTABLE_WORK': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'], 'correct':{'subj': ['PER','ORG'],'obj':'MISC'}},
    'AWARD_RECEIVED': {'subj':['MISC','PER','ORG'],'obj':['MISC','PER','ORG'], 'correct':{'subj': ['PER','MISC','ORG'],'obj':'MISC'}},
    'OWNED_BY': {"subj": ['MISC','ORG','PER','LOC'], 'obj': ['MISC','ORG','PER','LOC'], 'correct':{'subj': ['MISC','ORG','LOC'],'obj': ['PER','ORG']}},
    'CAST_MEMBER': {"subj": ['MISC','PER'], 'obj': ['MISC','PER'],'correct':{'subj': 'PER','obj':'MISC'}},
    'TERRITORY_CLAIMED_BY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}},
    'PERFORMER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct':{'subj': 'PER','obj':'MISC'}},
    'LOCATION_OF_FORMATION': {"subj": ['ORG','LOC'], 'obj': ['ORG','LOC'],'correct':{'subj': 'ORG','obj':'LOC'}},
    'COUNTRY_OF_ORIGIN': {"subj": ['LOC','ORG','MISC','PER'], 'obj': ['LOC','ORG','MISC','PER'], 'correct':{'subj': ['ORG','MISC','PER'],'obj':'LOC'}},
    'FATHER': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}},
    'MOTHER': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}},
    'HEAD_OF_STATE': {'subj': ['LOC','PER'],'obj':['LOC','PER'], 'correct':{'subj': 'LOC','obj':'PER'}},
    'CHILD': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}},
    'SPOUSE': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}},
    'START_TIME': {"subj": ['TIME','MISC'], 'obj': ['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'END_TIME': {'subj': ['TIME','MISC'],'obj':['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'POINT_IN_TIME': {'subj': ['TIME','MISC'],'obj':['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'ORIGINAL_NETWORK': {"subj": ['MISC',"ORG"], 'obj': ['MISC','ORG'],'correct':{'subj': 'MISC','obj':'ORG'}},
    'HAS_PART': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'], 'correct':{'subj': ['LOC','ORG','MISC'],'obj':['LOC','ORG','MISC']}},
    'DIRECTOR': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'SCREENWRITER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'CHARACTERS': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'PRESENT_IN_WORK': {'subj': ['PER','MISC'],'obj':['PER','MISC'], 'correct': {'subj':'PER','obj':'MISC'}},
    'CREATOR': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'EMPLOYER': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':['PER','ORG']}},
    'PRODUCER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'],'correct': {'subj':'MISC','obj':'PER'}},
    'ORIGINAL_LANGUAGE_OF_WORK': {"subj": ['MISC'], 'obj': ['MISC'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'DEVELOPER': {"subj": ['ORG','MISC','PER'], 'obj': ['ORG','MISC','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PUBLISHER': {"subj": ['ORG','MISC','PER'], 'obj': ['ORG','MISC','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PLATFORM': {"subj": ['MISC'], 'obj': ['MISC'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'PRODUCTION_COMPANY': {"subj": ['ORG','MISC'], 'obj': ['ORG','MISC'],'correct': {'subj':'MISC','obj':'ORG'}},
    'EDUCATED_AT': {"subj": ['PER','ORG','LOC'], 'obj': ['PER','ORG','LOC'], 'correct': {'subj':'PER','obj':['ORG','LOC']}},
    'OPERATOR': {"subj": ['PER','ORG','MISC'], 'obj': ['PER','ORG','MISC'], 'correct': {'subj':'MISC','obj':['ORG','PER']}},
    'LEAGUE': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': ['PER','ORG'],'obj':'ORG'}},
    'LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': ["LOC",'ORG'],'obj':'LOC'}},
    'LOCATION': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['MISC','LOC','ORG'],'obj':'LOC'}},
    'LOCATED_ON_TERRAIN_FEATURE': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['MISC','LOC','ORG'],'obj':'LOC'}},
    'RELIGION': {"subj": ['ORG','LOC','PER','MISC'], 'obj': ['ORG','LOC','PER','MISC'], 'correct':{'subj': ['PER','LOC','ORG'],'obj':'MISC'}},
    'LYRICS_BY': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct':{'subj': 'MISC','obj':'PER'}},
    'RECORD_LABEL': {"subj": ['ORG','MISC'], 'obj': ['ORG','MISC'], 'correct':{'subj': 'MISC','obj':'ORG'}},
    'MILITARY_BRANCH': {"subj": ['PER','ORG','MISC','LOC'], 'obj': ['PER','ORG','MISC','LOC'], 'correct':{'subj': ['PER','ORG','MISC','LOC'],'obj':'ORG'}},
    'CONFLICT': {'subj': ['MISC','PER'],'obj':['MISC','PER'],'correct':{'subj': ['MISC','PER'],'obj':'MISC'}},
    'MEMBER_OF_SPORTS_TEAM': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':'ORG'}},
    'LEGISLATIVE_BODY': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'], 'correct':{'subj': ['LOC','ORG','MISC'],'obj':'ORG'}}, # ??????,
    'WORK_LOCATION': {'subj': ['LOC','PER'],'obj':['LOC','PER'], 'correct':{'subj': 'PER','obj':'LOC'}},
    'APPLIES_TO_JURISDICTION': {'subj': ['LOC','ORG','MISC'],'obj':['LOC','ORG','MISC'], 'correct':{'subj': ['MISC','ORG'],'obj':'LOC'}},
    'PRODUCT_OR_MATERIAL_PRODUCED': {"subj": ['ORG','MISC'], 'obj': ['ORG','MISC'], 'correct': {'subj':'ORG','obj':'MISC'}},
    'GENRE': {"subj": ['MISC','PER'], 'obj': ['MISC','PER'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'COMPOSER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'MANUFACTURER': {"subj": ['MISC','ORG','PER'], 'obj': ['MISC','ORG','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PARTICIPANT_OF': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'], 'correct': {'subj':['PER','ORG'],'obj':'MISC'}},
    'FOUNDED_BY': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'],'correct': {'subj':['MISC','ORG'],'obj':'PER'}},
    'SUBSIDIARY': {"subj": ['ORG'], 'obj': ['ORG'], 'correct':{'subj': 'ORG','obj':'ORG'}},
    'CAPITAL' : {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}},
    'CAPITAL_OF': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}},
    'PARENT_ORGANIZATION': {"subj": ['ORG'], 'obj': ['ORG'], 'correct':{'subj': 'ORG','obj':'ORG'}},
    'ETHNIC_GROUP': {"subj": ['LOC','ORG','PER','MISC'], 'obj': ['LOC','ORG','PER','MISC'], 'correct':{'subj': ['LOC','ORG','PER'],'obj':['LOC','ORG','MISC']}},
    'BASIN_COUNTRY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': "LOC",'obj':'LOC'}},
    'OFFICIAL_LANGUAGE': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ["LOC",'ORG'],'obj': 'MISC'}},
    'MOUTH_OF_THE_WATERCOURSE': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': "LOC",'obj':'LOC'}},
    'NARRATIVE_LOCATION': {'subj': ['LOC','MISC'],'obj':['LOC','MISC'],'correct':{'subj': 'MISC','obj':'LOC'}},
    'INFLUENCED_BY': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'], 'correct':{'subj': ["PER",'ORG','MISC'],'obj': ["PER",'ORG','MISC']}},
    'RESIDENCE': {'subj': ['LOC','PER'],'obj':['LOC','PER'],'correct':{'subj': 'PER','obj':'LOC'}},
    'UNEMPLOYMENT_RATE': {"subj": ["LOC","TIME", "NUM"], "obj":  ["LOC","TIME","NUM"], 'correct':{'subj': 'LOC','obj':['TIME','NUM']}},
    'SISTER_CITY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}},
    'SEPARATED_FROM': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': ['LOC','ORG'],'obj':['LOC','ORG']}},
    'PARENT_TAXON': {"subj": ['LOC','MISC'], 'obj': ['LOC','MISC'], 'correct':{'subj': ['LOC','MISC'],'obj':['LOC','MISC']}},
    'REPLACES': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['LOC','MISC','ORG'],'obj':['LOC','MISC','ORG']}},
    'REPLACED_BY': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['LOC','MISC','ORG'],'obj':['LOC','MISC','ORG']}},
    'FOLLOWS': {"subj": ["MISC", "LOC", "ORG", "TIME", "PER"], 'obj': ["MISC", "LOC", "ORG", "TIME", "PER"], 'correct':{'subj': ["MISC", "LOC", "ORG", "TIME", "PER"],'obj':["MISC", "LOC", "ORG", "TIME", "PER"]}},
    'FOLLOWED_BY': {"subj": ["MISC", "LOC", "ORG", "TIME", "PER"], 'obj': ["MISC", "LOC", "ORG", "TIME", "PER"], 'correct':{'subj': ["MISC", "LOC", "ORG", "TIME", "PER"],'obj':["MISC", "LOC", "ORG", "TIME", "PER"]}},
    'INSTANCE_OF': {"subj": ['MISC','LOC','ORG','PER'], 'obj': ['MISC','LOC','ORG','PER'], 'correct':{'subj': ['MISC','LOC','ORG','PER'],'obj':['MISC','LOC','ORG','PER']}},  #check
    'CONTINENT': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'],'correct':{'subj': ['LOC','ORG','MISC'],'obj':'LOC'}},
    'MEMBER_OF': {"subj": ['PER','ORG','MISC'], 'obj': ['PER','ORG','MISC'],'correct':{'subj': 'PER','obj':['ORG','MISC']}},
    'PART_OF':{"subj": ['MISC','ORG','LOC'], 'obj': ['MISC','ORG','LOC'], 'correct':{'subj': ['MISC','ORG','LOC'],'obj':['ORG','MISC']}},
}


unknown_types = {                                     #?
    'SERIES': {"subj": ['?'], 'obj': ['?']},
    'SUBCLASS_OF':{"subj": ['?'], 'obj': ['?']},
}

In [ ]:
def filter_triples_by_type(triples):
    correct = []
    incorrect = []
    for triple in triples:
        if triple['pred'] not in correct_types.keys():
            incorrect.append(triple)
            continue
        types = correct_types[triple['pred']]
        if triple['subj'][1] in types['subj'] and triple['obj'][1] in types['obj']:
            correct.append(triple)
        else:
            incorrect.append(triple)
    print(f'Correct: {len(correct)} and Incorrect: {len(incorrect)}')
    return correct

In [ ]:
def filter_triples_by_correct_type(triples):
    correct_triples = []
    updated_triples = []
    incorrect_triples = []
    unruled_triples = []
    country_triples = []
    for triple in triples:
        if triple['pred'] not in correct_types.keys():
            unruled_triples.append(triple)
            continue
        types = correct_types[triple['pred']]
        correct_subj_types = types['correct']['subj']
        correct_obj_types = types['correct']['obj']
        # Оба типа не списки
        if not isinstance(correct_subj_types,list)and not isinstance(correct_obj_types,list):
            # Правильные типы не равны
            if correct_subj_types!=correct_obj_types:
                if triple['subj'][1] == correct_subj_types and triple['obj'][1] == correct_obj_types:
                    correct_triples.append(triple)
                elif triple['subj'][1] == correct_obj_types and triple['obj'][1] == correct_subj_types:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                else:
                    incorrect_triples.append(triple)
            # Правильные типы равны
            else:
                # subj = nlp(triple['subj'][0])
                # obj = nlp(triple['obj'][0])
                # pred = nlp(triple['pred'].replace('_',' '))
                # if subj.similarity(pred)>obj.similarity(pred):
                #     #TODO
                #     pass
                # else:
                #     #TODO
                #     pass
                unruled_triples.append(triple)
        # Хотя бы один из правильных типов - список
        if (isinstance(correct_subj_types,list)) or isinstance(correct_obj_types,list):
            # Субъект - список, а объект - не список
            if isinstance(correct_subj_types,list) and not isinstance(correct_obj_types,list):
                # Правильный тип объекта есть в списке правильных типов субъекта
                if correct_obj_types in correct_subj_types:
                    set_1_obj = set([correct_obj_types])
                    set_subj = set(correct_subj_types)
                    difference = set_subj.difference(set_1_obj)
                    if triple['subj'][1]!=triple['obj'][1]:
                        if triple['subj'][1] in difference and triple['obj'][1]==correct_obj_types:
                            correct_triples.append(triple)
                        elif triple['obj'][1] in difference and triple['subj'][1]==correct_obj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                        else:
                            incorrect_triples.append(triple)
                    else:
                        if triple['subj'][1] in difference:
                            incorrect_triples.append(triple)
                        else:
                            if triple['pred']=='COUNTRY':
                                subj = nlp(triple['subj'][0][0])
                                obj = nlp(triple['obj'][0][0])
                                pred = nlp(triple['pred'].lower())
                                if pred.similarity(subj)>pred.similarity(obj):
                                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                                    country_triples.append(correct_triple) #------------------
                                else:
                                    country_triples.append(triple)
                            else:
                                unruled_triples.append(triple)
                else:
                    if triple['obj'][1] == correct_obj_types:
                        if triple['subj'][1] in correct_subj_types:
                            correct_triples.append(triple)
                        else:
                            incorrect_triples.append(triple)
                    elif triple['subj'][1] == correct_obj_types:
                        if triple['obj'][1] in correct_subj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                        else:
                            incorrect_triples.append(triple)
                    else:
                        incorrect_triples.append(triple)
            # Субъект - не список, а объект - список
            if not isinstance(correct_subj_types,list) and isinstance(correct_obj_types,list):
                # Правильный тип субъекта есть в списке правильных типов объекта
                if correct_subj_types in correct_obj_types:
                    set_obj = set(correct_obj_types)
                    set_1_subj = set([correct_subj_types])
                    difference = set_obj.difference(set_1_subj)
                    if triple['subj'][1]!=triple['obj'][1]:
                        if triple['obj'][1] in difference and triple['subj'][1]==correct_subj_types:
                            correct_triples.append(triple)
                        elif triple['subj'][1] in difference and triple['obj'][1]==correct_subj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                        else:
                            incorrect_triples.append(triple)
                    else:
                        if triple['subj'][1] in difference:
                            incorrect_triples.append(triple)
                        else:
                            unruled_triples.append(triple)
                else:
                    # Тип субъекта - правильный
                    if triple['subj'][1] == correct_subj_types:
                        # Тип  объекта есть в списке правильных типов объекта
                        if triple['obj'][1] in correct_obj_types:
                            correct_triples.append(triple)
                        # Иначе
                        else:
                            incorrect_triples.append(triple)
                    # Тип субъекта не равен правильному типу субъекта, а тип объекта равен правильному типу субъекта -> меняем их местами
                    elif triple['obj'][1] == correct_subj_types:
                        if triple['subj'][1] in correct_obj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                        else:
                            incorrect_triples.append(triple)
                    else:
                        incorrect_triples.append(triple)

            if isinstance(correct_subj_types,list) and isinstance(correct_obj_types,list):
                set_subj = set(correct_subj_types)
                set_obj = set(correct_obj_types)
                only_subj = set_subj.difference(set_obj)
                only_obj = set_obj.difference(set_subj)
                both = set_subj.intersection(set_obj)
                if triple['subj'][1] in only_subj and triple['obj'][1] in only_obj:
                    correct_triples.append(triple)
                elif triple['subj'][1] in only_obj and triple['obj'][1] in only_subj:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                elif triple['subj'][1] in only_subj and triple['obj'][1] in both:
                    correct_triples.append(triple)
                elif triple['subj'][1] in only_obj and triple['obj'][1] in both:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                elif triple['subj'][1] in both and triple['obj'][1] in only_obj:
                    correct_triples.append(triple)
                elif triple['subj'][1] in both and triple['obj'][1] in only_subj:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                elif triple['subj'][1] in both and triple['obj'][1] in both:
                    #TODO
                    unruled_triples.append(triple)
                else:
                    incorrect_triples.append(triple)
    print('Correct: ',len(correct_triples))
    print('Incorrect: ',len(incorrect_triples))
    print('Updated: ',len(updated_triples))
    print('Unruled: ',len(unruled_triples))
    print('\n'.join(list(map(lambda x: str(x),country_triples))))
    return unruled_triples

In [ ]:
with open ('extracted_triples_2.json') as f:
    triples = json.load(f)

In [ ]:
print(len(triples))
correct_by_overall_type = filter_triples_by_type(triples)
unruled = filter_triples_by_correct_type(correct_by_overall_type)
pd.set_option('display.max_rows', None)

949
Correct: 909 and Incorrect: 40


<ipython-input-9-2aa0dd8a92d0>:62: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if pred.similarity(subj)>pred.similarity(obj):


Correct:  293
Incorrect:  68
Updated:  271
Unruled:  230
{'subj': [['s. u. v.'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['manhattan'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['collect pond park'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['america america'], 'LOC'], 'pred': 'COUNTRY', 'obj': [['america america'], 'LOC']}
{'subj': [['brownsville'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['texas'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['boston'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['capitol'], 'LOC'], 'obj': [['america'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['manhattan criminal courtroom'], 'LOC'], 'pred': 'COUNTRY', 'obj': [['the united states'], 'LOC']}
{'subj': [['squirrel hill'], 'LOC'], 'obj': [['american'], 'LOC'], 'pred': 'COUNTRY'}
{'subj': [['american'], 'LOC'], 'pred': 'COUNTRY', 'obj': [['tree of life synagogue']

In [ ]:
df = pd.DataFrame(triples)
df_un = pd.DataFrame(unruled)
print(df_un.groupby(by='pred').count())
print(df_un.loc[df_un['pred']=='COUNTRY'])

#print(df.loc[df['pred']=='OWNED_BY'])

                                                  subj  obj
pred                                                       
BASIN_COUNTRY                                        2    2
CHILD                                                8    8
CONTAINS_ADMINISTRATIVE_TERRITORIAL_ENTITY          30   30
CONTINENT                                            4    4
ETHNIC_GROUP                                         3    3
FATHER                                               7    7
FOLLOWED_BY                                          5    5
FOLLOWS                                              2    2
GENRE                                                1    1
HAS_PART                                            15   15
INSTANCE_OF                                          4    4
LEAGUE                                               2    2
LEGISLATIVE_BODY                                     1    1
LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER                  5    5
LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIA